# Deep Learning

Cargar librerías
Importa las librerías necesarias para procesamiento de imágenes, creación de la CNN y división de datos.

Definir carpeta de imágenes
Se indica la ruta donde están las imágenes de gatos y perros.

Cargar y procesar imágenes
Lee cada imagen .jpg, la redimensiona a 128x128, la normaliza y asigna etiqueta (0=gato, 1=perro).

Convertir a arrays
Convierte las listas de imágenes y etiquetas a arrays de numpy y aplica one-hot encoding a las etiquetas.

In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


folder = r"C:\Users\isaac\Downloads\dogs-vs-cats\dogs-vs-cats\train"

images = []
labels = []

for filename in os.listdir(folder):
    if filename.endswith(".jpg"):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (128,128))  # Reducción a 128x128
        images.append(img)
        
        if "cat" in filename.lower():
            labels.append(0)
        elif "dog" in filename.lower():
            labels.append(1)

images = np.array(images, dtype='float32') / 255.0
labels = to_categorical(np.array(labels), num_classes=2)



Se separan los datos en conjunto de entrenamiento (80%) y prueba (20%) manteniendo la proporción de clases (stratify).

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    images, labels, test_size=0.2, random_state=42, stratify=labels
)

Construcción del modelo CNN
Se crea una red convolucional con tres bloques de convolución y pooling, seguida de capas densas y Dropout para evitar sobreajuste. Se compila con optimizador Adam y función de pérdida categorical_crossentropy.

In [ ]:
model = Sequential()

# Bloque 1
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)))
model.add(MaxPool2D((2,2)))

# Bloque 2
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPool2D((2,2)))

# Bloque 3
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPool2D((2,2)))

# Clasificador
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Definición de callbacks

Se crean dos callbacks: ModelCheckpoint para guardar el mejor modelo según la validación, y EarlyStopping para detener el entrenamiento si la pérdida de validación no mejora durante 5 épocas.

In [ ]:
checkpoint = ModelCheckpoint("best_model_light.h5", monitor='val_accuracy', save_best_only=True, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


Entrenamiento del modelo

Se entrena la red neuronal con los datos de entrenamiento, usando un 20% para validación, hasta 20 épocas y con batch de 16, aplicando los callbacks definidos para guardar el mejor modelo y detenerse temprano si es necesario.

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=20,
    batch_size=16,
    callbacks=[checkpoint, early_stop]
)

Se calcula el rendimiento del modelo sobre el conjunto de prueba, mostrando la precisión (accuracy) alcanzada en datos no vistos.

In [ ]:
loss, acc = model.evaluate(X_test, y_test)
print(f"✅ Accuracy en test: {acc*100:.2f}%")

Se almacena el modelo entrenado en un archivo .h5 para poder cargarlo y usarlo posteriormente sin necesidad de reentrenar.

In [ ]:
model.save("final_dogs_vs_cats_light_model.h5")
print("Modelo guardado como final_dogs_vs_cats_light_model.h5")